In [1]:
import numpy as np
from scipy.io import loadmat
from matplotlib import pyplot as plt
import seaborn as sns
from scipy.signal import find_peaks
from scipy import signal
from scipy.optimize import curve_fit
import cmath
import control
from control.matlab import *
plt.style.use('seaborn-darkgrid')
from lmfit.models import LorentzianModel, QuadraticModel,LinearModel

# Lorentzian Decomposition
The current report shows how we extract the global lorentzian decomposition of the plate experiment. The goal is to get a first approximation of the main frequencies that excite the different modes, and subsequently model each spatial point of the plate with these known frequencies.

To achieve this, we will fit the absolute value of the sum of the spectral descomposition of every point in the plate and extract the frequencies.

### 1.0 Data Manipulation
Extracting the data, transforming it to the frequency domain and getting the sum

In [2]:
import hdf5storage
data = hdf5storage.loadmat('data/final_data.mat')

In [3]:
data.keys()

dict_keys(['XX', 'Xmatrix', 'YY', 'ZZ', 'f0', 'name2Save', 'sampleintervalS'])

In [7]:
1900-120

1780

In [34]:
fft_sum = 0
ffts = {}

Fc_low = 13500
Fc_high = 1000
L = 1880
Fs = 199999
colors = ['blue','green','orange','red']


import json
import datetime


for i in range(118):
    for j in range(118):
        
        #results[f'[{i},{j}]'] = {'params': [] , 'red_chi': [],'chi_sq': []}
        v = data['ZZ'][:,i,j][120:2000]

        sos_l = signal.bessel(5,Fc_low, 'low', fs=Fs,output='sos') # LPF filtering parameters
        sos_h = signal.bessel(5,Fc_high, 'high', fs=Fs,output='sos') # HPF filtering parameters

        x = signal.sosfilt(sos_l,v) # LPF
        x = signal.sosfilt(sos_h,x) # HPF

        freqs=np.fft.fftfreq(L,1/Fs)
        f = freqs[freqs>400] #frequency vector (x-axis in frec domain)
        fft= np.fft.fft(x)[freqs>400]
        
        ffts[f'{i},{j}'] = fft
        fft_sum = fft_sum + fft
            
            


In [108]:
len(ffts['74,33'])

936

In [20]:
%matplotlib qt
plt.plot(f,np.abs(fft_sum)/(117*10),'-o')
plt.plot(f,np.abs(ffts['74,33']),'-o')
index = [np.where(f>p)[0][0] for i,p in enumerate(p20)]
plt.plot(f[index],np.abs(fft_sum[index])/(117*10),'o')
plt.xlabel('Frequency [Hz]')
plt.ylabel('FFT (Abs)')

Text(0, 0.5, 'FFT (Abs)')

### 1.0 Looking at the maxima

In [105]:
max_ = [max(ffts[k]) for k in ffts]

a = np.array(max_)[np.array(max_)>30000]
print(f'Total points: {len(max_)}')
print(f'Points that passed the threshold: {len(a)}')
print(f'percentage of points that passed the threshold: {len(a)/len(max_) * 100} %')

Total points: 13924
Points that passed the threshold: 2507
percentage of points that passed the threshold: 18.004883654122377 %


### 2.0  Estimating the Peaks 
Here we apply some functions that give us a first estimation of the amount and places of the peaks. The documentation is in section 4.0

In [106]:
peaks = find_peaks_(f,np.abs(fft_sum))
peaks = find_peaks(np.abs(fft_sum))[0]
plt.plot(f[new_peaks],np.abs(fft_sum[new_peaks]),'o')
plt.plot(f[index],np.abs(fft_sum[index]),'.')
plt.plot(f,np.abs(fft_sum))

In [70]:
p2 = [5390,5500,6169,6450,6865,7089,7544,7970,8672,8882,9221,9670,10016,10426,10673,11013,11800,12490]

In [71]:
new_peaks = [np.where(f>p)[0][0] for p in p2]

### 3.0 Finding the best Fit
Curvefitting the dataset with our model

In [77]:
import matplotlib.pyplot as plt
from numpy import exp, loadtxt, pi, sqrt

from lmfit import Model

datx = f
#tryonh with different points
coord = '74,33'
daty = ffts[coord]
daty = fft_sum

    
mod = Model(MyLorentzian2)
pars = mod.make_params()
for i,p in enumerate(new_peaks):
    prefix = 'lz%d_' % (i+1)
    pars.add(name = f'{prefix}center', value = datx[p],min = min(datx),max = 13000)
    pars.add(name = f'{prefix}sigma',value =  5, min=0)
        #pars[prefix + 'amplitude'].set(daty[cen],min = min(daty),max = max(daty))
    pars.add(name = f'{prefix}amplitude_real', value = daty[p].real,min = min(daty.real),max = max(daty.real) + 1)
    pars.add(name = f'{prefix}amplitude_imag', value = daty[p].imag,min = min(daty.imag),max = max(daty.imag) + 1)
    


result = mod.fit(np.abs(daty), pars, x=datx)
params_ = list(result.values.values())
#print(result.fit_report())
%matplotlib qt
plt.figure()
plt.xlabel('Frequency [Hz]')
plt.ylabel('FFT (Abs)')
plt.title(f'Point: {coord}')
plt.plot(datx, np.abs(daty), 'bo-', label='Original Data (abs of sum)')
plt.plot(datx, result.best_fit, 'r-',label='Best fit')
#plt.plot(datx, result.best_fit, 'r-', label='best fit')
plt.legend(loc='best')
plt.show()

In [79]:
centers_new = [result.values[k] for k in result.values if 'center' in k]

In [101]:
print(centers_new)

[5412.808271713509, 5612.11038571605, 6324.503304154022, 6380.4417622063465, 6406.121505718675, 6861.694279189701, 7077.525028825343, 7586.608794535136, 8251.356047802785, 8557.913141831705, 8881.526339770082, 8981.335585820168, 9224.594123834742, 10274.944365581925, 10293.965950948092, 10346.69327582523, 12732.42370972809, 13000.0]


### 4.0 Fixin Centers

In [83]:
X = [25,72,54,98,51]
Y = [63,41,23,95,46]
type_ = ['strong','strong','weak','weak','banana']

results = []
datx = f

for i in range(5):
    print(f'{Y[i]},{X[i]}')
    daty = ffts[f'{Y[i]},{X[i]}']

    mod = Model(MyLorentzian2)
    pars = mod.make_params()
    for i in range(18):
        prefix = 'lz%d_' % (i+1)
        
        pars.add(name = f'{prefix}center', value = datx[new_peaks[i]],vary=False)
        mod.set_param_hint(f'{prefix}center', vary=False)
        pars.add(name = f'{prefix}sigma',value = 50, min=0)
            #pars[prefix + 'amplitude'].set(daty[cen],min = min(daty),max = max(daty))

        #index_cen = np.where(f>main_f[i])[0][0]
        pars.add(name = f'{prefix}amplitude_real', value = daty[new_peaks[i]].real)
        pars.add(name = f'{prefix}amplitude_imag', value = daty[new_peaks[i]].imag)



    result = mod.fit(np.abs(daty), pars, x=datx)
    
    results.append(result)

63,25
41,72
23,54
95,98
46,51


In [99]:
for i,res in enumerate(results):
    
    fit = res.best_fit
    daty = ffts[f'{Y[i]},{X[i]}']
    plt.figure()
    plt.xlabel('Frequency [Hz]')
    plt.title(f'Fit for {type_[i]} position')
    
    plt.plot(f,np.abs(daty),'bo-',label='Data')
    plt.plot(f,fit,'r-',label='Best Fit')
    for i in range(1,19):

        ps = [result.values[k] for k in result.values.keys() if f'lz{i}_' in k ]
        #plt.plot(f,np.abs(Lorentzian(f,*ps)),ls='dashed')

    plt.legend()

### 4.0 Functions 
These are the functions built to:

~ Clean the data: Discrete Fourirer Transform the data, Bandpass frequencies

~ Find Optimal amount of peaks: gets an initial guess of the peaks and discards the "noisy" ones, convolves the data with a gaussian function to smoothen it, and searches for the rest of the peaks.

~ Get the best fit of our Lorentzian Model

In [58]:
def fft_clean_data(ydata,Fs,Fc_low,Fc_high):
    """fft the data,bandpasses the frequencies in the chosen range"""
    # low pass filter
    L = len(ydata) 

    sos_l = signal.bessel(15,Fc_low, 'low', fs=Fs,output='sos') # LPF filtering parameters
    sos_h = signal.bessel(15,Fc_high, 'high', fs=Fs,output='sos') # HPF filtering parameters

    X = signal.sosfilt(sos_l,v) # LPF
    X = signal.sosfilt(sos_h,X) # HPF

    freqs=np.fft.fftfreq(L,1/Fs) #frequency vector (x-axis in frec domain)
    fft= np.fft.fft(X)# displacement band passes transform 
    #fftPhase = [cmath.phase(z) for z in np.fft.fft(v)]

    return freqs,fft

def norm(x,y):
    """euclidean distance"""
    return np.sqrt(x**2 + y**2)

def consecutive_peaks(peak_index):
    """returns in sublists the consecutive indexes"""
    sub_list = np.split(peak_index, np.where(np.diff(peak_index) > 1)[0] + 1)
    return sub_list
   
    
def find_near_peaks(x,y,peaks):
    """ gets the indexes of the peaks that are 'near' each other. 
    'near': the distance beaing a higher order than the average . 
    Can be modified depending on the noise of the data (as it creates micro-peaks)
    """
    near_peaks = []
    peak_dist = norm(np.diff(x[peaks]),np.diff(y[peaks]))  #distance between each consecutive peak
    avg_peak_dist = np.mean(peak_dist)*10 # 1 higher order than mean of distance between peaks 
    for i,peak in enumerate(peaks):
        if i == 0:
            dist_next = norm(np.diff([x[peak],x[peaks[i+1]]]),np.diff([y[peak],y[peaks[i+1]]]))
            if dist_next < avg_peak_dist:
                near_peaks.append(peak)
        elif i == len(peaks)-1:
            dist_prev = norm(np.diff([x[peak],x[peaks[i-1]]]),np.diff([y[peak],y[peaks[i-1]]]))
            if dist_prev < avg_peak_dist:
                near_peaks.append(peak)
        else:
            dist_next = norm(np.diff([x[peak],x[peaks[i+1]]]),np.diff([y[peak],y[peaks[i+1]]]))
            dist_prev = norm(np.diff([x[peak],x[peaks[i-1]]]),np.diff([y[peak],y[peaks[i-1]]]))
            if dist_next < avg_peak_dist or dist_prev < avg_peak_dist:
                near_peaks.append(peak)
    return near_peaks


def find_peaks_(x,y):
    """finds the peak with scipy function
    Saves the peaks of the original data that are an order higher than the noise.
    For the noise peaks, saves only the peaks of a convolution between a Gaussian distrib. with 10 std and the 
    original function.
    """
    peaks, _ = find_peaks(y)
    
    near_peaks = find_near_peaks(x,y,peaks)
    
    peaks_ = np.setdiff1d(peaks,near_peaks) # not near peaks

    gaussian = signal.windows.gaussian(len(y), std=10)
    
    convolution = signal.fftconvolve(y, gaussian, mode='same')

    peak_list, _ = find_peaks(convolution)

    peaks_ = np.append(peaks_,peak_list)
    
    peaks_ = [p for p in peaks_ if x[p] <= 20000] #deletes the high frequency peaks (higher than 20kHz)
    return peaks_



def add_peak(prefix, center, amplitude, sigma):
    """adds an additional lorentzian to the model. none parameters fixed"""
    peak = LorentzianModel(prefix=prefix)
    pars = Parameters()
    pars.add(name=prefix + 'amplitude', value=amplitude)
    pars.add(name=prefix + 'center',value = center)
    pars.add(name=prefix + 'sigma', value = sigma, min=0)
    pars.add(name=prefix + 'fwhm', expr=f'2.0000000*{prefix}sigma')
    pars.add(name=prefix + 'height', expr=f'0.3183099*{prefix}amplitude/max(1e-15, {prefix}sigma)')
    return peak, pars


def get_cen(out):
    ln = set([k.split('_')[0] for k in out])
    cen = []
    amp = []
    for l in ln:
        if np.abs(out[f'{l}_amplitude']) > 1:
            cen.append(out[f'{l}_center'])
            amp.append(out[f'{l}_amplitude'])
    return cen

def MyLorentzian2(x,lz1_center,lz1_sigma,lz1_amplitude_real,lz1_amplitude_imag,lz2_center,lz2_sigma,lz2_amplitude_real,lz2_amplitude_imag,lz3_center,lz3_sigma,lz3_amplitude_real,lz3_amplitude_imag,lz4_center,lz4_sigma,lz4_amplitude_real,lz4_amplitude_imag,lz5_center,lz5_sigma,lz5_amplitude_real,lz5_amplitude_imag,lz6_center,lz6_sigma,lz6_amplitude_real,lz6_amplitude_imag,lz7_center,lz7_sigma,lz7_amplitude_real,lz7_amplitude_imag,lz8_center,lz8_sigma,lz8_amplitude_real,lz8_amplitude_imag,lz9_center,lz9_sigma,lz9_amplitude_real,lz9_amplitude_imag,lz10_center,lz10_sigma,lz10_amplitude_real,lz10_amplitude_imag,lz11_center,lz11_sigma,lz11_amplitude_real,lz11_amplitude_imag,lz12_center,lz12_sigma,lz12_amplitude_real,lz12_amplitude_imag,lz13_center,lz13_sigma,lz13_amplitude_real,lz13_amplitude_imag,lz14_center,lz14_sigma,lz14_amplitude_real,lz14_amplitude_imag,lz15_center,lz15_sigma,lz15_amplitude_real,lz15_amplitude_imag,lz16_center,lz16_sigma,lz16_amplitude_real,lz16_amplitude_imag,lz17_center,lz17_sigma,lz17_amplitude_real,lz17_amplitude_imag,lz18_center,lz18_sigma,lz18_amplitude_real,lz18_amplitude_imag):
    """returns the absolute values of the sum of 13 lorentzian function"""
    
    lz1_amplitude =  lz1_amplitude_real + 1j*lz1_amplitude_imag
    lz2_amplitude =  lz2_amplitude_real + 1j*lz2_amplitude_imag
    lz3_amplitude =  lz3_amplitude_real + 1j*lz3_amplitude_imag
    lz4_amplitude =  lz4_amplitude_real + 1j*lz4_amplitude_imag
    lz5_amplitude =  lz5_amplitude_real + 1j*lz5_amplitude_imag
    lz6_amplitude =  lz6_amplitude_real + 1j*lz6_amplitude_imag
    lz7_amplitude =  lz7_amplitude_real + 1j*lz7_amplitude_imag
    lz8_amplitude =  lz8_amplitude_real + 1j*lz8_amplitude_imag
    lz9_amplitude =  lz9_amplitude_real + 1j*lz9_amplitude_imag
    lz10_amplitude =  lz10_amplitude_real + 1j*lz10_amplitude_imag
    lz11_amplitude =  lz11_amplitude_real + 1j*lz11_amplitude_imag
    lz12_amplitude =  lz12_amplitude_real + 1j*lz12_amplitude_imag
    lz13_amplitude =  lz13_amplitude_real + 1j*lz13_amplitude_imag
    lz14_amplitude =  lz14_amplitude_real + 1j*lz14_amplitude_imag
    lz15_amplitude =  lz15_amplitude_real + 1j*lz15_amplitude_imag
    lz16_amplitude =  lz16_amplitude_real + 1j*lz16_amplitude_imag
    lz17_amplitude =  lz17_amplitude_real + 1j*lz17_amplitude_imag
    lz18_amplitude =  lz18_amplitude_real + 1j*lz18_amplitude_imag


    l1 = (lz1_amplitude * lz1_sigma**2) / ( lz1_sigma**2 + ( x - lz1_center )**2) 
    l2 = (lz2_amplitude * lz2_sigma**2) / ( lz2_sigma**2 + ( x - lz2_center )**2) 
    l3 = (lz3_amplitude * lz3_sigma**2) / ( lz3_sigma**2 + ( x - lz3_center )**2)
    l4 = (lz4_amplitude * lz4_sigma**2) / ( lz4_sigma**2 + ( x - lz4_center )**2) 
    l5 = (lz5_amplitude * lz5_sigma**2) / ( lz5_sigma**2 + ( x - lz5_center )**2)
    l6 = (lz6_amplitude * lz6_sigma**2) / ( lz6_sigma**2 + ( x - lz6_center )**2) 
    l7 = (lz7_amplitude * lz7_sigma**2) / ( lz7_sigma**2 + ( x - lz7_center )**2) 
    l8 = (lz8_amplitude * lz8_sigma**2) / ( lz8_sigma**2 + ( x - lz8_center )**2)
    l9 = (lz9_amplitude * lz9_sigma**2) / ( lz9_sigma**2 + ( x - lz9_center )**2) 
    l10 = (lz10_amplitude * lz10_sigma**2) / ( lz10_sigma**2 + ( x - lz10_center )**2) 
    l11 = (lz11_amplitude * lz11_sigma**2) / ( lz11_sigma**2 + ( x - lz11_center )**2) 
    l12 = (lz12_amplitude * lz12_sigma**2) / ( lz12_sigma**2 + ( x - lz12_center )**2) 
    l13 = (lz13_amplitude * lz13_sigma**2) / ( lz13_sigma**2 + ( x - lz13_center )**2)
    l14 = (lz14_amplitude * lz14_sigma**2) / ( lz14_sigma**2 + ( x - lz14_center )**2) 
    l15 = (lz15_amplitude * lz15_sigma**2) / ( lz15_sigma**2 + ( x - lz15_center )**2)
    l16 = (lz16_amplitude * lz16_sigma**2) / ( lz16_sigma**2 + ( x - lz16_center )**2) 
    l17 = (lz17_amplitude * lz17_sigma**2) / ( lz17_sigma**2 + ( x - lz17_center )**2) 
    l18 = (lz18_amplitude * lz18_sigma**2) / ( lz18_sigma**2 + ( x - lz18_center )**2)
    
    return np.abs(l1+l2+l3+l4+l5+l6+l7+l8+l9+l10+l11+l12+l13+l14+l15+l16+l17+l18)

def Lorentzian(x,lz1_center,lz1_sigma,lz1_amplitude_real,lz1_amplitude_imag):
    """returns the absolute values of the sum of 13 lorentzian function"""
    
    lz1_amplitude =  lz1_amplitude_real + 1j*lz1_amplitude_imag

    l1 = (lz1_amplitude * lz1_sigma**2) / ( lz1_sigma**2 + ( x - lz1_center )**2) 

    
    return l1


In [5]:
Fs = 199999 #sample freq 
Fc_Low = 18000 # cutting frequency low pass filter
Fc_High = 4000 # cutting frequency high pass filter
fft_sum = 0
for i in range(31):
    for j in range(31):
        v = data[:,i,j]
        f, fft = fft_clean_data(v,Fs,Fc_Low,Fc_High)
        fft_sum = fft_sum + fft
L = len(v)
f = f[10:L//2] 
fft = fft[10:L//2]      

### Functions (more)
For future use: fixing the known frequencies to each point

In [116]:

# based on https://lmfit.github.io/lmfit-py/builtin_models.html#lorentzianmodel
from lmfit import Model, Parameters



def lorentzians_optimization(datx,daty,main_frequencies,main_amplitudes):
    %matplotlib qt
    mod_i = []
    for i, frec in enumerate(main_frequencies):
        #frec_index = datx.index(frec)
        if i ==0:
            peak, pars = add_peak_('lz%d_' % (i+1), frec ,main_amplitudes[i],5)
            print(pars)
            params = peak.make_params(pars)
        else:
            peak, pars = add_peak_('lz%d_' % (i+1), frec ,main_amplitudes[i],5)
            params.update(pars)
        mod_i.append(peak)

    mod = mod_i[0]
    mod_i.pop(0)
    for i in mod_i:
        mod = mod + i

    init = mod.eval(params, x=datx)
    out = mod.fit(daty, params, x=datx)

    print(out.fit_report(min_correl=0.5))

    fig, axes = plt.subplots(1, 2, figsize=(12.8, 4.8))
    axes[0].plot(datx, daty, 'b',label='Original Data')
    axes[0].plot(datx, init, 'k--', label='Initial fit')
    axes[0].plot(datx, out.best_fit, 'r-', label='Best fit')
    axes[0].legend(loc='best')

    axes = axes.flatten()
    comps = out.eval_components(x=datx)
    axes[1].plot(datx, daty, 'b')
    for i, cen in enumerate(main_frequencies):
        axes[1].plot(datx, comps['lz%d_' % (i+1)], '--', label=f'Lorentzian {i}')
    plt.legend()
    plt.show()
    
    return out


def lorentzian( x, x0, a, gam ,x1,a1,gam1):
    l = (a * gam**2 / ( gam**2 + ( x - x0 )**2) )+ (a1 * gam1**2 / ( gam1**2 + ( x - x1 )**2))
    l_noisy = l+np.random.normal(0,0.01,x.shape)
    return l_noisy